In [1]:
%load_ext autoreload
%autoreload 2

import os
from tqdm import tqdm

from processingmm import batch_processing
from freezingeffect import selection_of_ROIs, automatic_ROI_propagation, collect_data_propagated
from freezingeffect.helpers import load_parameters_ROIs

## 1. Process the Mueller Matrices of the folders

In [2]:
# set the parameters to run the script
data_folder_path = r'F:\ImpactHistology\Freezing_dev'
directories = [data_folder_path]
calib_directory = r'F:\polarimetry\calibration'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'PD')

0it [00:00, ?it/s]


[]

## 2. Specify the folders that will be processed and clean them (remove precedent processing results)

In [3]:
path_folder_temp, path_align_folder = selection_of_ROIs.create_alignment_folder()
base_dirs = selection_of_ROIs.get_the_base_dirs(data_folder_path)
selection_of_ROIs.clean_the_alignment_folders(path_align_folder)
selection_of_ROIs.create_the_masks(data_folder_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:16<00:00,  1.38s/it]


## 3. Loop over all the folders that will be processed

#### 3.1. loop over all the folders that needs to be processed

In [4]:
from datetime import datetime
now = datetime.now()

In [5]:
now

datetime.datetime(2023, 6, 30, 17, 23, 16, 439998)

In [6]:
now.strftime("%d/%m/%Y %H:%M:%S")

'30/06/2023 17:23:16'

In [ ]:
# tissue_types = ['WM', 'GM']
tissue_types = ['WM', 'GM']
for path_folder in tqdm(base_dirs):
    propagation_lists = {}
    
    for tissue_type in tissue_types:
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_lists[tissue_type] = selection_of_ROIs.square_selection(path_folder_temp, path_folder, WM, 
                                                                            tissue_type, path_align_folder)
       
    # 3. actually do the alignment
    propagation_lists = automatic_ROI_propagation.generate_combined_mask(propagation_lists, path_align_folder)
    automatic_ROI_propagation.do_alignment(path_align_folder)
    output_folders = automatic_ROI_propagation.move_computed_folders(path_align_folder)
    
    # 4. and collect the data after propagation
    for tissue_type in tissue_types:
        WM = tissue_type == 'WM'
        collect_data_propagated.collect_data_propagated(WM, path_align_folder, propagation_lists[tissue_type], output_folders)

 17%|█████████████▊                                                                     | 1/6 [04:15<21:17, 255.57s/it]

## 4. Compare the parameters

#### 4.1. Get the folders for the measurements of interest

In [ ]:
from freezingeffect import parameter_comparaison

In [ ]:
# get the parameters for the comparaison
path_folder = data_folder_path
wavelength, match_sequence, measurement, max_, max_nb_of_squares = parameter_comparaison.get_parameters()
max_nb_of_squares = max_nb_of_squares * len(base_dirs)


# and the folders containing the data
data_folders, base_folders, to_be_removed = parameter_comparaison.get_data_folders(path_folder, wavelength, match_sequence)

data_folders[0:5] + data_folders[-5:]

#### 4.2. Get the data for the measurement

In [ ]:
data_types, data_fixation = parameter_comparaison.get_data(data_folders)

data_fixation['GM']['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2']

In [ ]:
# remove the ROI moving from white matter to grey matter and vice versa
for data_type, dfed in data_fixation.items():
    for folder, df in dfed.items():
        to_remove_dfed = to_be_removed[folder.split('\\polarimetry')[0]]
        for val in to_remove_dfed:
            for fol in val:
                if fol[0] == folder.split('\\')[-1].split('_align')[0]:
                    df = df.apply(parameter_comparaison.find_match_seq, match_seq = fol[1], axis = 1)
                    data_fixation[data_type][folder] = df

In [ ]:
data_fixation['GM']['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2']

In [ ]:
#### Set up the parameters to be studied
parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth']
metrics = ['median', 'mean', 'max']
times = ['T0', 'T1']
    
# create the directories that will be used to store the output of the analysis
folder = parameter_comparaison.create_output_directories(measurement, data_types, parameters = parameters)

In [ ]:
### Get the data and save it into xlsx file
to_combine = parameter_comparaison.get_and_organize_data(data_fixation, metrics, data_types, times, folder, max_,
                          parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth'])

to_combine[('retardance', 'WM', 'median')]

In [ ]:
to_combine_cv = {}
for idx, val in to_combine.items():
    if idx[0] == 'azimuth':
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times), azimuth = True)
    else:
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times))
        
to_combine_cv[('retardance', 'WM', 'median')]

#### 4.3. Statistical analysis

In [ ]:
#### Get the data to be used for the comparisons
# recombine the data in a different format
data_all_recombined = parameter_comparaison.recombine_data_tests(to_combine, times)
data_all_recombined_cv = parameter_comparaison.recombine_data_tests(to_combine_cv, times)

In [ ]:
#### Comparaison of the means and the means of the fold change
# perform the statistical test for both fold change and mean values
paired_test = parameter_comparaison.t_test(data_all_recombined, times, 'median')# times[1:]])
paired_test_cv = parameter_comparaison.t_test(data_all_recombined_cv, times, 'median')# times[1:]])
df_grouped = parameter_comparaison.create_test_df(paired_test, parameter = 'median')
df_grouped_cv = parameter_comparaison.create_test_df(paired_test_cv, parameter = 'median')
df_grouped_cv

In [ ]:
# finally, save the results
path_statistical_test = os.path.join('results', measurement)
df_grouped.to_excel(os.path.join(path_statistical_test, 'statistical_test.xlsx'))
df_grouped_cv.to_excel(os.path.join(path_statistical_test, 'statistical_test_cv.xlsx'))

#### 4.4. Combine the results in a single excel file

In [ ]:
# this cell allows to combine the data obtained for both grey and white matter in a table that can then be used
# to plot the results using Graphpad Prism
retardance = {}
depolarization = {}
for key, param in to_combine.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization[key[1]] = param
        
retardance_cv = {}
depolarization_cv = {}
azimuth_cv = {}
for key, param in to_combine_cv.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance_cv[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization_cv[key[1]] = param
    else:
        if key[2] == 'mean':
            azimuth_cv[key[1]] = param
        
retardance = parameter_comparaison.create_df_prism(retardance, max_nb_of_squares)
depolarization = parameter_comparaison.create_df_prism(depolarization, max_nb_of_squares)
    
retardance_cv = parameter_comparaison.create_df_prism(retardance_cv, max_nb_of_squares)
depolarization_cv = parameter_comparaison.create_df_prism(depolarization_cv, max_nb_of_squares)
azimuth_cv = parameter_comparaison.create_df_prism(azimuth_cv, max_nb_of_squares)

retardance_cv

In [ ]:
# finally save all the tables
path_data_prism = os.path.join('results', measurement)
retardance.to_excel(os.path.join(path_data_prism, "retardance_prism.xlsx"))
depolarization.to_excel(os.path.join(path_data_prism, "depolarization_prism.xlsx"))
retardance_cv.to_excel(os.path.join(path_data_prism, "retardance_prism_cv.xlsx"))
depolarization_cv.to_excel(os.path.join(path_data_prism, "depolarization_prism_cv.xlsx"))
azimuth_cv.to_excel(os.path.join(path_data_prism, "azimuth_prism_cv.xlsx"))